In [1]:
import numpy as np
import pandas as pd 
import requests as re
import time 

In [2]:
import datetime
today = datetime.date.today()

In [3]:
# 整理今日日期
year = str(int(str(today)[0:4])-1911)
month = str(today)[5:7]
day = str(today)[8:10]
todate = year + '/' + month +'/' + day
print(todate)

110/03/20


# 取得資料

## 取得上櫃外資淨買超並做整理

In [4]:
# get data 上櫃外資買超
gbdata = pd.read_html("https://www.tpex.org.tw/web/stock/3insti/qfii_trading/forgtr_result.php?l=zh-tw&t=D&type=buy&d=%s&s=0,asc&o=htm"%todate)
df = gbdata[0]
need = [df.columns[1], df.columns[2], df.columns[11]] #取出我要的東西

In [5]:
gbuydata = df[need] #把資料換成dataframe
gbuydata.columns[0][0]

'代號'

In [6]:
gbuydata.columns = ['代號','名稱','買超']
for i in range(len(gbuydata.index)):
    if len(gbuydata["代號"][i])>4:
        gbuydata=gbuydata.drop(index=i)

gbuydata.reset_index(drop=True)

,代號,名稱,買超
0,共0筆,共0筆,共0筆


## 取得上櫃外資賣超

In [207]:
gsdata = pd.read_html("https://www.tpex.org.tw/web/stock/3insti/qfii_trading/forgtr_result.php?l=zh-tw&t=D&type=sell&d=%s&s=0,asc&o=htm"%(todate))
df1 = gsdata[0]
need1 = [df1.columns[1], df1.columns[2], df1.columns[11]] #取出我要的東西

In [208]:
gselldata = df1[need1] #把資料換成dataframe
gselldata.columns[0][0]

'03月18日 外資及陸資賣超'

In [209]:
gselldata.columns = ['代號','名稱','賣超']
for i in range(len(gselldata.index)):
    if len(gselldata["代號"][i])>4:
        gselldata=gselldata.drop(index=i)

gselldata.reset_index(drop=True)

,代號,名稱,賣超
0,6219,富旺,-3652
1,5820,日盛金,-3554
2,8358,金居,-2186
3,6244,茂迪,-1035
4,6182,合晶,-1020
...,...,...,...
268,6629,泰金-KY,-1
269,8047,星雲,-1
270,8071,能率網通,-1
271,8431,匯鑽科,-1


## 取得上市外資買賣超

In [7]:
#上市外資買賣
#cdate = str(today)[0:4]+str(today)[5:7]+str(today)[8:10]
cdate = "20210319"
data2  = pd.read_html("https://www.twse.com.tw/fund/TWT38U?response=html&date=%s"%cdate)

In [8]:
df2 = data2[0]
need2 = [df2.columns[1], df2.columns[2], df2.columns[11]] #取出我要的東西

alldata = df2[need2] #把資料換成dataframe
alldata.columns[0][0]

'110年03月19日 外資及陸資買賣超彙總表'

In [9]:
#分買超跟賣超
alldata.columns = ['代號','名稱','買賣超']
for i in range(len(alldata.index)):
    if len(alldata["代號"][i])>4:
        alldata=alldata.drop(index=i)

alldata = alldata.reset_index(drop=True)

In [10]:
indb = alldata.loc[alldata.iloc[:,2]>0].index #取得index的數字再放到loc裡
inds = alldata.loc[alldata.iloc[:,2]<0].index

buydata = alldata.iloc[indb,:].reset_index(drop = True) #外資淨買超
selldata = alldata.iloc[inds,:].reset_index(drop =True) #外資淨賣超
buydata

,代號,名稱,買賣超
0,2353,宏碁,73388327
1,2883,開發金,58906038
2,2610,華航,32759506
3,2890,永豐金,32527364
4,2618,長榮航,18205175
...,...,...,...
255,3563,牧德,2000
256,2429,銘旺科,1000
257,8443,阿瘦,1000
258,8499,鼎炫-KY,1000


# 取得上市投信買超

In [12]:
def buy(x):
    #上市外資買賣
    data2  = pd.read_html(x)
    df2 = data2[0]
    need2 = [df2.columns[1], df2.columns[2], df2.columns[5]] #取出我要的東西
    alldata = df2[need2] #把資料換成dataframe
    alldata.columns = ['代號','名稱','買賣超']
    for i in range(len(alldata.index)):
        if len(alldata["代號"][i])>4:
            alldata=alldata.drop(index=i)

    alldata = alldata.reset_index(drop=True)
    indb = alldata.loc[alldata.iloc[:,2]>0].index #取得index的數字再放到loc裡
    inds = alldata.loc[alldata.iloc[:,2]<0].index

    buydata = alldata.iloc[indb,:].reset_index(drop = True) #外資淨買超
    selldata = alldata.iloc[inds,:].reset_index(drop =True) #外資淨賣超
    
    return buydata 

def sell(x):
    #上市外資買賣
    data2  = pd.read_html(x)
    df2 = data2[0]
    need2 = [df2.columns[1], df2.columns[2], df2.columns[5]] #取出我要的東西
    alldata = df2[need2] #把資料換成dataframe
    alldata.columns = ['代號','名稱','買賣超']
    for i in range(len(alldata.index)):
        if len(alldata["代號"][i])>4:
            alldata=alldata.drop(index=i)

    alldata = alldata.reset_index(drop=True)
    inds = alldata.loc[alldata.iloc[:,2]<0].index    
    selldata = alldata.iloc[inds,:].reset_index(drop =True) #外資淨賣超
    
    return selldata

In [14]:
#cdate = str(today)[0:4]+str(today)[5:7]+str(today)[8:10]
cdate = "20210319"
toxing="https://www.twse.com.tw/fund/TWT44U?response=html&date=%s"%cdate
foreign="https://www.twse.com.tw/fund/TWT38U?response=html&date=%s"%cdate
sell(toxing)

TypeError: object of type 'numpy.int64' has no len()

# 寫成函數並依照投信外資分類 之後好合併

In [15]:
# cdate = str(today)[0:4]+str(today)[5:7]+str(today)[8:10] #當天日期直接讀取
cdate = "20210318"

toxing="https://www.twse.com.tw/fund/TWT44U?response=html&date=%s"%cdate
foreign="https://www.twse.com.tw/fund/TWT38U?response=html&date=%s"%cdate

def buy(x):
    #上市外資買賣
    data2  = pd.read_html(x)
    df2 = data2[0]
    need2 = [df2.columns[1], df2.columns[2], df2.columns[5]] #取出我要的東西
    alldata = df2[need2] #把資料換成dataframe
    if "44" in x:
        alldata.columns = ['代號','名稱','投信買超']
    else:
        alldata.columns = ['代號','名稱','外資買超']
    for i in range(len(alldata.index)):
        if len(alldata["代號"][i])>4:
            alldata=alldata.drop(index=i)

    alldata = alldata.reset_index(drop=True)
    indb = alldata.loc[alldata.iloc[:,2]>0].index #取得index的數字再放到loc裡

    buydata = alldata.iloc[indb,:].reset_index(drop = True) #外資淨買超

    return buydata 

def sell(x):
    #上市外資買賣
    data2  = pd.read_html(x)
    df2 = data2[0]
    need2 = [df2.columns[1], df2.columns[2], df2.columns[5]] #取出我要的東西
    alldata = df2[need2] #把資料換成dataframe
    if "44" in x:
        alldata.columns = ['代號','名稱','投信賣超']
    else:
        alldata.columns = ['代號','名稱','外資賣超']
    
    for i in range(len(alldata.index)):
        if len(alldata["代號"][i])>4:
            alldata=alldata.drop(index=i)

    alldata = alldata.reset_index(drop=True)
    inds = alldata.loc[alldata.iloc[:,2]<0].index    
    selldata = alldata.iloc[inds,:].reset_index(drop =True) #外資淨賣超
    
    return selldata

In [16]:
buythemost = pd.merge(buy(toxing),buy(foreign))
buythemost['合計']=buythemost['投信買超']+buythemost['外資買超']
buythemost.sort_values('合計',ascending=False).reset_index(drop = True)

,代號,名稱,投信買超,外資買超,合計
0,2603,長榮,1306000,39507940,40813940
1,2317,鴻海,1595000,31769111,33364111
2,2409,友達,934000,20971906,21905906
3,2353,宏碁,46000,12154992,12200992
4,2609,陽明,178000,10495900,10673900
...,...,...,...,...,...
77,3653,健策,11000,30000,41000
78,9917,中保科,3000,27500,30500
79,1707,葡萄王,7000,19100,26100
80,1315,達新,4000,22000,26000


# 整合買賣的函式

In [23]:
# cdate = str(today)[0:4]+str(today)[5:7]+str(today)[8:10] #當天日期直接讀取
cdate = "20210318"

toxing="https://www.twse.com.tw/fund/TWT44U?response=html&date=%s"%cdate
foreign="https://www.twse.com.tw/fund/TWT38U?response=html&date=%s"%cdate

def tw(torf, bors):
    data2  = pd.read_html(torf)
    df2 = data2[0]
    need2 = [df2.columns[1], df2.columns[2], df2.columns[5]] #取出我要的東西
    alldata = df2[need2] #把資料換成dataframe
    if "44" in torf:
        alldata.columns = ['代號','名稱','投信賣超']
    else:
        alldata.columns = ['代號','名稱','外資賣超']
    
    for i in range(len(alldata.index)):
        if len(alldata["代號"][i])>4:
            alldata=alldata.drop(index=i)
            
    alldata = alldata.reset_index(drop=True)
    
    if bors=="s":
        inds = alldata.loc[alldata.iloc[:,2]<0].index    
        selldata = alldata.iloc[inds,:].reset_index(drop =True) #外資淨賣超
        
        return selldata
    
    else:
        indb = alldata.loc[alldata.iloc[:,2]>0].index #取得index的數字再放到loc裡
        buydata = alldata.iloc[indb,:].reset_index(drop = True) #外資淨買超
        
        return buydata
    

In [28]:
tw(foreign,'b')

,代號,名稱,外資賣超
0,2603,長榮,39507940
1,2317,鴻海,31769111
2,3576,聯合再生,31577700
3,2409,友達,20971906
4,2353,宏碁,12154992
...,...,...,...
532,8467,波力-KY,1000
533,8940,新天地,1000
534,9926,新海,1000
535,9930,中聯資源,1000


# 上市投信renew

In [15]:
df3 = pd.read_html("https://www.twse.com.tw/fund/TWT44U?response=html&date=20210319")

In [19]:
data3 = df3[0]

In [41]:
need = [data3.columns[1],data3.columns[2],data3.columns[5]]

In [42]:
need

[('110年03月19日 投信買賣超彙總表', '證券代號'),
 ('110年03月19日 投信買賣超彙總表', '證券名稱'),
 ('110年03月19日 投信買賣超彙總表', '買賣超股數')]

In [44]:
ddf = data3[need]

In [45]:
ddf.columns = ['代號','名稱','買賣超']

In [46]:
ddf

,代號,名稱,買賣超
0,2603,長榮,12468000
1,2610,華航,8097000
2,2317,鴻海,5492464
3,3481,群創,5045000
4,2891,中信金,2310730
...,...,...,...
206,2059,川湖,-1000
207,2101,南港,-1000
208,2201,裕隆,-1000
209,9904,寶成,0


In [56]:
for i in range(len(ddf.index)):
    if len(str(ddf['代號'][i]))>4:
        print(ddf['代號'][i])

In [57]:
len(str(ddf["代號"][0]))

4